# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Init

In [3]:
import datahandler

import matplotlib.pyplot as plt


dataset_id = "oslo"

data_preprocessor = datahandler.DataPreprocessorOUS(dataset_id)
data_preprocessor.execute()

data_loader = datahandler.DataLoader(dataset_id)
data_loader.execute()


Loading dataset: 100%|██████████| 6/6 [01:07<00:00, 11.32s/it]


# Analysis
- All of the rows where "time_available" is NaN should be dropped (64 rows)
- All of the rows where "time_dispatch" is NaN should be dropped (111 rows)
- All of the rows where "triage_impression_during_call" is NaN should be dropped (9 rows)
- All of the rows where "time_arrival_scene" is NaN and "time_arrival_hospital" is not NaN should be dropped (13695 rows)
- All of the rows where "time_departure_scene" is NaN and "time_arrival_hospital" is not NaN should be dropped (8356 rows)


In [4]:
nan_rows = data_loader.processed_incidents_df[data_loader.processed_incidents_df["time_available"].isna()]
print(nan_rows.shape[0])


64


In [5]:
nan_rows = data_loader.processed_incidents_df[data_loader.processed_incidents_df["time_dispatch"].isna()]
print(nan_rows.shape[0])

111


In [6]:
nan_rows = data_loader.processed_incidents_df[data_loader.processed_incidents_df["triage_impression_during_call"].isna()]
print(nan_rows.shape[0])

9


In [7]:
count = data_loader.processed_incidents_df[(data_loader.processed_incidents_df["time_arrival_scene"].isna()) & (~data_loader.processed_incidents_df["time_arrival_hospital"].isna())].shape[0]
print(count)

13695


In [8]:
count = data_loader.processed_incidents_df[(data_loader.processed_incidents_df["time_departure_scene"].isna()) & (~data_loader.processed_incidents_df["time_arrival_hospital"].isna())].shape[0]
print(count)

8356
